<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import folium
import requests
import pandas as pd
import os
import webbrowser

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [3]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(spacex_csv_file)

In [4]:
spacex_df.groupby(["Launch Site"])["Flight Number"].count()

Launch Site
CCAFS LC-40     26
CCAFS SLC-40     7
KSC LC-39A      13
VAFB SLC-4E     10
Name: Flight Number, dtype: int64

Now, you can take a look at what are the coordinates for each site.


In [5]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df
   

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [6]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map1 = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [7]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='blue', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map1.add_child(circle)
site_map1.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [8]:
# Initialize the map centered at the NASA coordinate, with an appropriate zoom level
site_map2 = folium.Map(location=nasa_coordinate, zoom_start=5)


In [9]:

# Create a FeatureGroup to group all launch zone markers and circles
launch_zones = folium.map.FeatureGroup(name="Launch Zones")
# Usamos itertuples para extraer los datos de cada fila del DataFrame de manera eficiente.
# itertuples() devuelve un iterador de tuplas nombradas, donde cada tupla representa una fila.
# Esto es más rápido que iterrows() y permite acceder a los valores de las columnas como atributos.
# En este caso, extraemos el nombre del sitio, la latitud y la longitud de cada fila.
coordinates = [(row._1, row.Lat, row.Long) for row in launch_sites_df.itertuples()]

# Iterate over the coordinate list to create markers and circle overlays
for name, lat, long in coordinates:
    
    # Create a circle to mark the launch zone
    circle = folium.Circle(
        location=(lat, long),     # Set the circle's position
        radius=1000,              # Set the radius in meters
        color='Red',              # Outline color
        #fill_color='Blue',        # Fill color inside the circle
        #fill=True                 # Enable fill
    ).add_child(folium.Popup(f"Launch Site: {name}"))  # Add popup with site name
    
    marker = folium.map.Marker(              # Crear un marcador de etiqueta con el nombre del sitio de lanzamiento usando DivIcon
		location=(lat, long),  # Ubicación del marcador (latitud, longitud)
		icon=DivIcon(
			icon_size=(150, 150),  # El primer valor (20) es el ancho en píxeles, el segundo valor (10) es la altura en píxeles del área del icono
			icon_anchor=(0, 0),    # Punto de anclaje del icono
			html=f'<div style="font-size: 12px; color:#d35400;"><b>{name}</b></div>'  # HTML para mostrar el nombre del sitio
		)
	)

    # Add both the circle and the label marker to the FeatureGroup
    launch_zones.add_child(circle)
    launch_zones.add_child(marker)

# Add the FeatureGroup (with all zones) to the main map
site_map2.add_child(launch_zones)

# Optional: Add layer control if you plan to toggle layers (can be useful in complex maps)
# site_map.add_child(folium.LayerControl())

# # 7. Guardar el mapa en HTML
# output_file = "site_map1.html"
# site_map2.save(output_file)
# print(f"✅ Mapa guardado como '{output_file}'.")

# import os
# import webbrowser

# # 8. Abrir en navegador (ruta absoluta)
# full_path = os.path.realpath(output_file)
# webbrowser.open(f"file://{full_path}")

# Add the FeatureGroup (with all zones) to the main map
site_map2.add_child(launch_zones)

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [10]:
marker_cluster = MarkerCluster()


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [11]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)


,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [12]:
# Iterate over each row in the spacex_df DataFrame.
# This loop processes each launch record, one row at a time.
for index, record in spacex_df.iterrows():
    
    # Create a string for the popup that indicates the launch status.
    # We check if 'marker_color' is 'green' to determine success; otherwise, it's treated as failed.
    popup_text = f"Launch Status: {'Success' if record['marker_color'] == 'green' else 'Failed'}"
    
    # Create a Marker for each record using the specific latitude and longitude.
    # The popup is created using folium.Popup:
    #   - The popup_text variable is passed to provide dynamic content.
    #   - parse_html=True enables the popup to correctly parse and render any HTML content, allowing for richer formatting if needed.
    # The Marker also customizes the icon:
    #   - icon='rocket' uses a Font Awesome rocket icon, which is visually relevant for SpaceX launches.
    #   - color is dynamically set using record['marker_color'] to indicate success (e.g., green) or failure (e.g., red).
    #   - prefix='fa' specifies that the icon comes from the Font Awesome library.
    marker = folium.Marker(
        location=(record['Lat'], record['Long']),
        popup=folium.Popup(popup_text, parse_html=True),  # parse_html=True ensures HTML content inside the popup is rendered correctly.
        icon=folium.Icon(icon='rocket', color=record['marker_color'], prefix='fa')
    )
    
    # Add the customized marker to the MarkerCluster.
    # MarkerCluster groups nearby markers together for a cleaner map display and improved user experience.
    marker_cluster.add_child(marker)





# Add the entire MarkerCluster (with all its markers) to the site_map.
# This step ensures that the clustered markers are rendered on the final map.
site_map2.add_child(marker_cluster)




In [13]:
# 7. Guardar el mapa en HTML
output_file = "site_map2.html"
site_map2.save(output_file)
print(f"✅ Mapa guardado como '{output_file}'.")



# 8. Abrir en navegador (ruta absoluta)
full_path = os.path.realpath(output_file)
webbrowser.open(f"file://{full_path}")


✅ Mapa guardado como 'site_map2.html'.


True

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [14]:
# Define a formatter function as a string to format the latitude and longitude values
# The function rounds the numbers to 5 decimal places for a cleaner display.
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

# Create a MousePosition control to display the coordinates (Lat, Long) as the mouse moves over the map.
# This control is useful for debugging and user interactivity, showing precise map coordinates.
mouse_position = MousePosition(
    # Set the location of the control on the map. 'topright' positions the control at the top right corner.
    position='topright',
    
    # Specify a separator string between the longitude and other parts of the displayed text.
    # Here it separates the longitude value from the rest of the label with ' Long: '.
    separator=' Long: ',
    
    # Provide a value to display when no coordinates are available. 'NaN' indicates not a number.
    empty_string='NaN',
    
    # Control whether the longitude appears first in the output. Setting to False will show latitude first.
    lng_first=False,
    
    # Set the maximum number of digits for displaying the coordinate numbers.
    num_digits=20,
    
    # Prefix text that appears before the latitude value.
    prefix='Lat:',
    
    # Provide a function to format the latitude values. The formatter rounds to 5 decimal places.
    lat_formatter=formatter,
    
    # Provide a function to format the longitude values. The same formatter is used for consistency.
    lng_formatter=formatter,
)

# Add the mouse_position control to the site_map.
# This ensures that as a user moves their mouse over the map, the coordinates will be displayed according to the defined settings.
site_map2.add_child(mouse_position)

# Output the map object (site_map) so it can be rendered.
site_map2


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [15]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [16]:
# find coordinate of the closet coastline
launch_site_lat= 28.5623  
launch_site_lon=-80.57736
coastline_lat= 28.56274
coastline_lon=-80.56773

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

0.9420529314743292

_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [17]:

coordinate = (coastline_lat,coastline_lon)
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 

distance_marker = folium.Marker(
	coordinate,
	icon=DivIcon(
		icon_size=(20, 20),
		icon_anchor=(0, 0),
		html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline),
	)
)






_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [18]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# Draw a PolyLine between the launch site and the coastline point
line = folium.PolyLine(locations=[(launch_site_lat, launch_site_lon), coordinate],weight=2,color='blue')

site_map2.add_child(line)

# lines=folium.PolyLine(locations=coordinates, weight=1)

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [19]:


# --- Asunciones (deben existir previamente en tu código) ---
# Necesitas definir estas variables antes de usarlas. Ejemplo:
launch_site_lat = 28.5623  # Latitud de ejemplo para el sitio de lanzamiento (LC-40)
launch_site_lon = -80.5774 # Longitud de ejemplo para el sitio de lanzamiento (LC-40)

# # Necesitas definir la función calculate_distance. Ejemplo:
# def calculate_distance(lat1, lon1, lat2, lon2):
#     R = 6371 # Radio de la Tierra en km
#     dLat = math.radians(lat2 - lat1)
#     dLon = math.radians(lon2 - lon1)
#     lat1 = math.radians(lat1)
#     lat2 = math.radians(lat2)
#     a = math.sin(dLat/2)**2 + math.cos(lat1)*math.cos(lat2)*math.sin(dLon/2)**2
#     c = 2*math.atan2(math.sqrt(a), math.sqrt(1-a))
#     return R * c

# Necesitas inicializar el mapa folium. Ejemplo:
site_map2 = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=14) # Ajustado zoom para mejor vista
# --- Fin de Asunciones ---


# --- Ciudad: Melbourne, FL ---
# Define las coordenadas de la ciudad.
city_lat = 28.0836    # Latitud para Melbourne, FL.
city_lon = -80.6081   # Longitud para Melbourne, FL.

# Calcula la distancia entre el sitio de lanzamiento y la ciudad.
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# Crea un marcador para la ciudad con la distancia.
city_marker = folium.Marker(
    (city_lat, city_lon),   # Posición del marcador.
    icon=DivIcon(
        icon_size=(150, 30), # Tamaño ajustado para visibilidad de texto
        icon_anchor=(0, 0),   # Ancla del icono (esquina superior izquierda).
        # Contenido HTML: muestra la distancia formateada y "KM City".
        html='<div style="font-size: 18px; color:#d35400; font-weight:bold;">{:.2f} KM City</div>'.format(distance_city),
    )
)
# Añade el marcador de la ciudad al mapa.
site_map2.add_child(city_marker)

# Dibuja una línea conectando el sitio de lanzamiento y la ciudad.
city_line = folium.PolyLine(
    locations=[(launch_site_lat, launch_site_lon), (city_lat, city_lon)],
    weight=2,      # Grosor de la línea.
    color='purple' # Color de la línea.
)
# Añade la línea de la ciudad al mapa.
site_map2.add_child(city_line)


# --- Ferrocarril más cercano ---
# Define las coordenadas del punto del ferrocarril.
rail_lat = 28.5631    # Latitud del ferrocarril.
rail_lon = -80.58702  # Longitud del ferrocarril.

# Calcula la distancia desde el sitio de lanzamiento al ferrocarril.
distance_rail = calculate_distance(launch_site_lat, launch_site_lon, rail_lat, rail_lon)

# Crea un marcador para el ferrocarril con la distancia.
rail_marker = folium.Marker(
    (rail_lat, rail_lon),  # Posición del marcador.
    icon=DivIcon(
        icon_size=(150, 30), # Tamaño ajustado
        icon_anchor=(0, 0),  # Ancla del icono.
        # Contenido HTML: muestra la distancia formateada y "KM Rail".
        html='<div style="font-size: 18px; color:#d35400; font-weight:bold;">{:.2f} KM Rail</div>'.format(distance_rail),
    )
)
# Añade el marcador del ferrocarril al mapa.
site_map2.add_child(rail_marker)

# Dibuja una línea entre el sitio de lanzamiento y el ferrocarril.
rail_line = folium.PolyLine(
    locations=[(launch_site_lat, launch_site_lon), (rail_lat, rail_lon)],
    weight=2,    # Grosor de la línea.
    color='green'# Color de la línea.
)
# Añade la línea del ferrocarril al mapa.
site_map2.add_child(rail_line)


# --- Autopista más cercana ---
# Define las coordenadas del punto de la autopista.
highway_lat = 28.56342   # Latitud de la autopista.
highway_lon = -80.57085  # Longitud de la autopista.

# Calcula la distancia desde el sitio de lanzamiento a la autopista.
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Crea un marcador para la autopista con la distancia.
highway_marker = folium.Marker(
    (highway_lat, highway_lon),  # Posición del marcador.
    icon=DivIcon(
        icon_size=(150, 30), # Tamaño ajustado
        icon_anchor=(0, 0),  # Ancla del icono.
        # Contenido HTML: muestra la distancia formateada y "KM Hwy".
        # CORRECCIÓN: Se ajustó font-size de 50 a 16px para consistencia.
        html='<div style="font-size: 18px; color:#d35400; font-weight:bold;">{:.2f} KM Hwy</div>'.format(distance_highway),
    )
)
# Añade el marcador de la autopista al mapa.
site_map2.add_child(highway_marker)

# Dibuja una línea conectando el sitio de lanzamiento y la autopista.
highway_line = folium.PolyLine(
    locations=[(launch_site_lat, launch_site_lon), (highway_lat, highway_lon)],
    weight=2,     # Grosor de la línea.
    color='orange'# Color de la línea.
)
# Añade la línea de la autopista al mapa.
site_map2.add_child(highway_line)


# --- Costa más cercana --- NUEVA SECCIÓN ---
# Define las coordenadas de un punto en la costa cercana.
# (Estas coordenadas son una estimación, ajústalas si es necesario)
coast_lat = 28.56342   # Latitud de la costa (similar a la autopista pero más al este).
coast_lon = -80.5675   # Longitud de la costa (ligeramente al este de la autopista).

# Calcula la distancia desde el sitio de lanzamiento a la costa.
distance_coast = calculate_distance(launch_site_lat, launch_site_lon, coast_lat, coast_lon)

# Crea un marcador para la costa con la distancia.
coast_marker = folium.Marker(
    (coast_lat, coast_lon),  # Posición del marcador.
    icon=DivIcon(
        icon_size=(150, 30), # Tamaño ajustado
        icon_anchor=(0, 0),  # Ancla del icono.
        # Contenido HTML: muestra la distancia formateada y "KM Coast".
        html='<div style="font-size: 18px; color:#d35400; font-weight:bold;">{:.2f} KM Coast</div>'.format(distance_coast),
    )
)
# Añade el marcador de la costa al mapa.
site_map2.add_child(coast_marker)

# Dibuja una línea conectando el sitio de lanzamiento y la costa.
coast_line = folium.PolyLine(
    locations=[(launch_site_lat, launch_site_lon), (coast_lat, coast_lon)],
    weight=2,   # Grosor de la línea.
    color='blue' # Color de la línea (azul para distinguirla).
)
# Añade la línea de la costa al mapa.
site_map2.add_child(coast_line)


# --- Guardar y Abrir el Mapa ---
# Define el nombre del archivo de salida HTML.
output_file = "site_map_with_coast.html"
# Guarda el mapa en el archivo HTML.
site_map2.save(output_file)
print(f"✅ Mapa guardado como '{output_file}'.")

# Obtiene la ruta absoluta del archivo guardado.
full_path = os.path.realpath(output_file)
# Abre el archivo HTML en el navegador web predeterminado.
webbrowser.open(f"file://{full_path}")

site_map2

✅ Mapa guardado como 'site_map_with_coast.html'.


In [20]:
full_path

'/Users/luciddreams/Library/Mobile Documents/com~apple~CloudDocs/Programas/IBM Datascience/Practice_Projects/Proyecto Final/site_map_with_coast.html'

After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
- Are launch sites in close proximity to highways?
- Are launch sites in close proximity to coastline?
- Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


In [21]:
print(f'''
1. Are the launch sites in close proximity to railroads?
    Yes, the distance to the nearest railroad is approximately {distance_rail:.2f} km.

2. Are the launch sites in close proximity to highways?
    Yes, the distance to the nearest highway is approximately {distance_highway:.2f} km.

3. Are the launch sites in close proximity to the coast?
    Yes, the distance to the nearest coastline is approximately {distance_coastline:.2f} km.

4. Do the launch sites maintain a certain distance from cities?
    Yes, the distance to the nearest city is approximately {distance_city:.2f} km.

Explanation:
The launch sites are strategically located near key infrastructures such as railroads, highways, and the coast to facilitate logistics and safety, while maintaining a prudent distance from cities to minimize risks to the population.
''')



1. Are the launch sites in close proximity to railroads?
    Yes, the distance to the nearest railroad is approximately 0.94 km.

2. Are the launch sites in close proximity to highways?
    Yes, the distance to the nearest highway is approximately 0.65 km.

3. Are the launch sites in close proximity to the coast?
    Yes, the distance to the nearest coastline is approximately 0.94 km.

4. Do the launch sites maintain a certain distance from cities?
    Yes, the distance to the nearest city is approximately 53.33 km.

Explanation:
The launch sites are strategically located near key infrastructures such as railroads, highways, and the coast to facilitate logistics and safety, while maintaining a prudent distance from cities to minimize risks to the population.



# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
